In [1]:
import pandas as pd
import findspark
import pyspark
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrameReader
from pyspark.sql.functions import *
findspark.init()

In [2]:
spark = SparkSession.builder.getOrCreate() #create a spark session

In [5]:
import psycopg2 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
con = psycopg2.connect(database='ecommerce',user='postgres',password='123');
cursor = con.cursor()
with open(r'C:\Users\hp\Desktop\projects+Resume and essentials\Real-time product analtytic tool\product.csv', 'rb') as f: #reading the saved processed file to copy it's contents to the database table

    next(f)

    cursor.copy_from(f,'ecommerce', sep=",") #copy contents of file to the table

    con.commit()

In [14]:


df=spark.read.format("csv").option("header","true").option("inferSchema","true").load('product.csv') #convert .csv file to pyspark dataframe
    

In [3]:
df.head(5)

[Row(order_id='cfcd208495d565ef66e7dff9f98764da', user_id='c81e728d9d4c2f636f067f89cc14862c', page_id='6f4922f45568161a8cdf4ad2299f6d23', product='sneakers', site_version='desktop', time=datetime.datetime(2019, 1, 11, 9, 24, 43), title='banner_click', target=0),
 Row(order_id='c4ca4238a0b923820dcc509a6f75849b', user_id='eccbc87e4b5ce2fe28308fd9f2a7baf3', page_id='4e732ced3463d06de0ca9a15b6153677', product='sneakers', site_version='desktop', time=datetime.datetime(2019, 1, 9, 9, 38, 51), title='banner_show', target=0),
 Row(order_id='c81e728d9d4c2f636f067f89cc14862c', user_id='eccbc87e4b5ce2fe28308fd9f2a7baf3', page_id='5c45a86277b8bf17bff6011be5cfb1b9', product='sports_nutrition', site_version='desktop', time=datetime.datetime(2019, 1, 9, 9, 12, 45), title='banner_show', target=0),
 Row(order_id='eccbc87e4b5ce2fe28308fd9f2a7baf3', user_id='eccbc87e4b5ce2fe28308fd9f2a7baf3', page_id='fb339ad311d50a229e497085aad219c7', product='company', site_version='desktop', time=datetime.datetime(201

In [8]:
df.columns

['order_id',
 'user_id',
 'page_id',
 'product',
 'site_version',
 'time',
 'title',
 'target']

In [15]:
df.count()

1048575

In [3]:
df.groupBy('order_id').count().sort(desc("count")).distinct().head(5) #order_id is unique

[Row(order_id='0000023f507999464aa2b78875b7e5d6', count=1),
 Row(order_id='000009891526c0ade7180f8423792063', count=1),
 Row(order_id='00000ce845c00cbf0686c992fc369df4', count=1),
 Row(order_id='00000f7264c27ba6fea0c837ed6aa0aa', count=1),
 Row(order_id='0000104cd168386a335ba6bf6e32219d', count=1)]

In [8]:
df.groupBy('product').count().sort(desc("count")).distinct().head(5) #top 5 products

[Row(product='clothes', count=1786438),
 Row(product='company', count=1725056),
 Row(product='sneakers', count=1703342),
 Row(product='sports_nutrition', count=1634625),
 Row(product='accessories', count=1621759)]

In [5]:
df.groupBy('user_id').count().sort(desc("count")).distinct().head(5) #top 5 users

[Row(user_id='6e1d9fca53a19d4f105947348fd1d627', count=23),
 Row(user_id='27af71559f2723868c078c49bd3ca0db', count=22),
 Row(user_id='2b9a39dddd5b7a18891dfd0d3aa3c599', count=22),
 Row(user_id='b89dfb3b9656f8836849bcb874afac70', count=22),
 Row(user_id='aa33d045826d234eb5858674dbfe93f2', count=22)]

In [6]:
df.groupBy('page_id').count().sort(desc("count")).distinct().head(5)

[Row(page_id='a3d2de7675556553a5f08e4c88d2c228', count=248722),
 Row(page_id='f218ebcda49e5da7ac8b4e51aed47550', count=2),
 Row(page_id='e6d8545daa42d5ced125a4bf747b3688', count=2),
 Row(page_id='a64dd9fe8a80e90bc5e0ec85303953bb', count=2),
 Row(page_id='d324a0cc02881779dcda44a675fdcaaa', count=2)]

In [9]:
df = df.withColumn('product', regexp_replace('product', 'company', 'electronics'))

In [11]:
df.head(5)

[Row(order_id='cfcd208495d565ef66e7dff9f98764da', user_id='c81e728d9d4c2f636f067f89cc14862c', page_id='6f4922f45568161a8cdf4ad2299f6d23', product='sneakers', site_version='desktop', time=datetime.datetime(2019, 1, 11, 9, 24, 43), title='banner_click', target=0),
 Row(order_id='c4ca4238a0b923820dcc509a6f75849b', user_id='eccbc87e4b5ce2fe28308fd9f2a7baf3', page_id='4e732ced3463d06de0ca9a15b6153677', product='sneakers', site_version='desktop', time=datetime.datetime(2019, 1, 9, 9, 38, 51), title='banner_show', target=0),
 Row(order_id='c81e728d9d4c2f636f067f89cc14862c', user_id='eccbc87e4b5ce2fe28308fd9f2a7baf3', page_id='5c45a86277b8bf17bff6011be5cfb1b9', product='sports_nutrition', site_version='desktop', time=datetime.datetime(2019, 1, 9, 9, 12, 45), title='banner_show', target=0),
 Row(order_id='eccbc87e4b5ce2fe28308fd9f2a7baf3', user_id='eccbc87e4b5ce2fe28308fd9f2a7baf3', page_id='fb339ad311d50a229e497085aad219c7', product='electronics', site_version='desktop', time=datetime.datetime